In [1]:
import torch

In [2]:
from audiocraft.data.midi_dataset import EnCodecTokenMIDIDataset

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.8.18 (you have 3.8.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


/home/jongmin/.local/share/virtualenvs/audiocraft-ssbSDm-j/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
paths_loaded = torch.load('/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/maestro-v3.0.0_split.pt')

In [6]:
dataset = EnCodecTokenMIDIDataset(paths_loaded['validation'])

  3%|▎         | 4/137 [00:00<00:05, 22.80it/s]

Length mismatch! Popping -2 index
/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID--AUDIO_12_R1_2004_03_Track03_wav--1.midi
39 38
Length mismatch! Popping -2 index
/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2004/MIDI-Unprocessed_SMF_16_R1_2004_01-08_ORIG_MID--AUDIO_16_R1_2004_03_Track03_wav.midi
12 11


  9%|▉         | 13/137 [00:00<00:05, 21.00it/s]

Length mismatch! Popping -2 index
/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2004/MIDI-Unprocessed_XP_11_R1_2004_03-04_ORIG_MID--AUDIO_11_R1_2004_04_Track04_wav.midi
20 19


 35%|███▌      | 48/137 [00:02<00:03, 28.80it/s]

...

Length mismatch! Popping -2 index
/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2011/MIDI-Unprocessed_16_R1_2011_MID--AUDIO_R1-D6_15_Track15_wav.midi
13 12


 72%|███████▏  | 99/137 [00:04<00:01, 22.29it/s]

Length mismatch! Popping -2 index
/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_01_R1_2015_wav--2.midi
11 10
Length mismatch! Popping -2 index
/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_03_R1_2015_wav--4.midi
19 18


100%|██████████| 137/137 [00:05<00:00, 24.04it/s]

Length mismatch! Popping -2 index
/home/jongmin/userdata/MAESTRO/maestro-v3.0.0/2018/MIDI-Unprocessed_Recital9-11_MID--AUDIO_09_R1_2018_wav--4.midi
10 9


In [7]:
len(dataset)

137

In [8]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [9]:
from audiocraft.modules.semcodec import SemCodecMidiDecoder

In [10]:
model = SemCodecMidiDecoder().to("cuda")

In [11]:
input, target_pr = next(iter(dataloader))

In [12]:
out = model(input.to("cuda"))

In [15]:
out.shape

torch.Size([8, 2, 88, 1500])

In [18]:
qq = extract_notes(out[0,0].T,out[0,1].T,None)

In [19]:
qq

(array([ 1,  3,  7, ..., 77, 80, 82]),
 array([[   0,    3],
        [   0,    1],
        [   0,    4],
        ...,
        [1499, 1500],
        [1499, 1500],
        [1499, 1500]]))

In [13]:
import numpy as np
import torch


def extract_notes(onsets, frames, velocity, onset_threshold=0.5, frame_threshold=0.5):
    """
    Finds the note timings based on the onsets and frames information

    Parameters
    ----------
    onsets: torch.FloatTensor, shape = [frames, bins]
    frames: torch.FloatTensor, shape = [frames, bins]
    velocity: torch.FloatTensor, shape = [frames, bins]
    onset_threshold: float
    frame_threshold: float

    Returns
    -------
    pitches: np.ndarray of bin_indices
    intervals: np.ndarray of rows containing (onset_index, offset_index)
    velocities: np.ndarray of velocity values
    """
    onsets = (onsets > onset_threshold).cpu().to(torch.uint8)
    frames = (frames > frame_threshold).cpu().to(torch.uint8)
    onset_diff = torch.cat([onsets[:1, :], onsets[1:, :] - onsets[:-1, :]], dim=0) == 1

    pitches = []
    intervals = []
    # velocities = []

    for nonzero in onset_diff.nonzero():
        frame = nonzero[0].item()
        pitch = nonzero[1].item()

        onset = frame
        offset = frame
        # velocity_samples = []

        while onsets[offset, pitch].item() or frames[offset, pitch].item():
            # if onsets[offset, pitch].item():
                # velocity_samples.append(velocity[offset, pitch].item())
            offset += 1
            if offset == onsets.shape[0]:
                break

        if offset > onset:
            pitches.append(pitch)
            intervals.append([onset, offset])
            # velocities.append(np.mean(velocity_samples) if len(velocity_samples) > 0 else 0)

    return np.array(pitches), np.array(intervals) #, np.array(velocities)


def notes_to_frames(pitches, intervals, shape):
    """
    Takes lists specifying notes sequences and return

    Parameters
    ----------
    pitches: list of pitch bin indices
    intervals: list of [onset, offset] ranges of bin indices
    shape: the shape of the original piano roll, [n_frames, n_bins]

    Returns
    -------
    time: np.ndarray containing the frame indices
    freqs: list of np.ndarray, each containing the frequency bin indices
    """
    roll = np.zeros(tuple(shape))
    for pitch, (onset, offset) in zip(pitches, intervals):
        roll[onset:offset, pitch] = 1

    time = np.arange(roll.shape[0])
    freqs = [roll[t, :].nonzero()[0] for t in time]
    return time, freqs

In [14]:
import sys
from collections import defaultdict

import numpy as np
from mir_eval.multipitch import evaluate as evaluate_frames
from mir_eval.transcription import precision_recall_f1_overlap as evaluate_notes
from mir_eval.transcription_velocity import precision_recall_f1_overlap as evaluate_notes_with_velocity
from mir_eval.util import midi_to_hz
from scipy.stats import hmean
from tqdm import tqdm

SAMPLE_RATE = 32000
HOP_LENGTH = 640
MIN_MIDI = 21
MAX_MIDI = 108

eps = sys.float_info.epsilon


def evaluate(pred, label, onset_threshold=0.5, frame_threshold=0.5):
    metrics = defaultdict(list)

    pred = torch.nn.functional.relu(pred)

    p_ref, i_ref = extract_notes(label[0].T, label[1].T, None)
    p_est, i_est = extract_notes(pred[0].T, pred[1].T, None, onset_threshold, frame_threshold)

    t_ref, f_ref = notes_to_frames(p_ref, i_ref, label[1].shape)
    t_est, f_est = notes_to_frames(p_est, i_est, pred[1].shape)

    scaling = HOP_LENGTH / SAMPLE_RATE

    i_ref = (i_ref * scaling).reshape(-1, 2)
    p_ref = np.array([midi_to_hz(MIN_MIDI + midi) for midi in p_ref])
    i_est = (i_est * scaling).reshape(-1, 2)
    p_est = np.array([midi_to_hz(MIN_MIDI + midi) for midi in p_est])

    t_ref = t_ref.astype(np.float64) * scaling
    f_ref = [np.array([midi_to_hz(MIN_MIDI + midi) for midi in freqs]) for freqs in f_ref]
    t_est = t_est.astype(np.float64) * scaling
    f_est = [np.array([midi_to_hz(MIN_MIDI + midi) for midi in freqs]) for freqs in f_est]

    p, r, f, o = evaluate_notes(i_ref, p_ref, i_est, p_est, offset_ratio=None)
    metrics['metric/note/precision'].append(p)
    metrics['metric/note/recall'].append(r)
    metrics['metric/note/f1'].append(f)
    metrics['metric/note/overlap'].append(o)

    p, r, f, o = evaluate_notes(i_ref, p_ref, i_est, p_est)
    metrics['metric/note-with-offsets/precision'].append(p)
    metrics['metric/note-with-offsets/recall'].append(r)
    metrics['metric/note-with-offsets/f1'].append(f)
    metrics['metric/note-with-offsets/overlap'].append(o)

    frame_metrics = evaluate_frames(t_ref, f_ref, t_est, f_est)
    metrics['metric/frame/f1'].append(hmean([frame_metrics['Precision'] + eps, frame_metrics['Recall'] + eps]) - eps)

    for key, loss in frame_metrics.items():
        metrics['metric/frame/' + key.lower().replace(' ', '_')].append(loss)

    return metrics

In [14]:
out[0,0].shape

torch.Size([88, 1500])

In [17]:
import warnings
warnings.filterwarnings('default')

In [32]:
i = 7
a = evaluate(out[i], target_pr[i])
a

defaultdict(list,
            {'metric/note/precision': [0.0035287541052337365],
             'metric/note/recall': [0.8706896551724138],
             'metric/note/f1': [0.007029020808685365],
             'metric/note/overlap': [0.13458018355039242],
             'metric/note-with-offsets/precision': [6.987631891551953e-05],
             'metric/note-with-offsets/recall': [0.017241379310344827],
             'metric/note-with-offsets/f1': [0.00013918853086505672],
             'metric/note-with-offsets/overlap': [0.8754480286738355],
             'metric/frame/f1': [0.10490753911806561],
             'metric/frame/precision': [0.05635148042024833],
             'metric/frame/recall': [0.7583547557840618],
             'metric/frame/accuracy': [0.05535747795083505],
             'metric/frame/substitution_error': [0.2416452442159383],
             'metric/frame/miss_error': [0.0],
             'metric/frame/false_alarm_error': [12.45758354755784],
             'metric/frame/total_error

In [16]:
a

defaultdict(list,
            {'metric/note/precision': [0.003123635508682363],
             'metric/note/recall': [0.8454545454545455],
             'metric/note/f1': [0.00622427467121775],
             'metric/note/overlap': [0.1902006336988417],
             'metric/note-with-offsets/precision': [0.0004366372216437712],
             'metric/note-with-offsets/recall': [0.11818181818181818],
             'metric/note-with-offsets/f1': [0.0008700599002777498],
             'metric/note-with-offsets/overlap': [0.6670681882220355],
             'metric/frame/f1': [0.05838876570583906],
             'metric/frame/precision': [0.030291411042944784],
             'metric/frame/recall': [0.8061224489795918],
             'metric/frame/accuracy': [0.030072325846973735],
             'metric/frame/substitution_error': [0.19387755102040816],
             'metric/frame/miss_error': [0.0],
             'metric/frame/false_alarm_error': [25.612244897959183],
             'metric/frame/total_error'